In [1]:
import pandas as pd
import json
pd.set_option('display.max_columns', None)

In [2]:
# person_df = pd.read_csv('./data/person.csv')
caregiver_df = pd.read_csv('./data/caregiver.csv', dtype=str)
helpee_df = pd.read_csv('./data/helpee.csv', dtype=str)
address_df = pd.read_csv('./data/address.csv', dtype=str)
relationship_df = pd.read_csv('./data/relationship.csv', dtype=str)
caregiver_address_df = (
    caregiver_df.merge(
        address_df.rename(
            {
                "id": "address_id"
            },
            axis=1,
        ),
        on="address_id",
        suffixes=["_caregiver", "_address"],    
    )
)
response = caregiver_address_df.to_dict(orient="records")
response
caregiver_address_df

,caregiver_id,name,photo_url,birth_date,gender,occupation,address_id,user_id,experience,mental_load,helper_bio,longitude,latitude,number,street,zip_code,city
0,1,Pierre,NaN,19850427,M,active,1,NaN,new,low,"Je suis Pierre, très beau, et m'occupe de ma m...",48.78602961813197,2.283740146208942,55,rue la Fontaine,92260,Fontenay-aux-Roses
1,2,Marc,NaN,19850815,M,active,3,NaN,medium,high,Je m'appelle Marc et je suis déplaisant à rega...,43.519119711543965,5.462731655722142,67,cours Gambetta,13100,Aix en Provence


In [3]:
helpee_add_df = helpee_df.merge(
        address_df,
        on="address_id",
        suffixes=["_caregiver", "_address"],    
    )
helpee_add_df

,helpee_id,name,photo_url,birth_date,gender,occupation,address_id,disease,longitude,latitude,number,street,zip_code,city
0,1,Daniele,NaN,19570315,F,retired,2,Maman poule,43.70153201752642,3.8064635557728814,104,rue des amours d'antan,34980,Saint Gély du Fesc
1,2,Jean-Marc,NaN,19540101,M,retired,4,Papa poulet,49.013887388856425,6.384466569400809,19,l'orée du bois,57580,Rémilly


In [4]:
helpee_columns = list(set(list(helpee_add_df.columns) + list(relationship_df.columns)))
helpee_columns

['since',
 'name',
 'address_id',
 'latitude',
 'gender',
 'city',
 'longitude',
 'birth_date',
 'zip_code',
 'caregiver_id',
 'number',
 'disease',
 'relationship',
 'photo_url',
 'street',
 'helpee_id',
 'helpee_bio',
 'occupation']

In [5]:
helpee_columns = {
    'helpee_id': 'helpee_id',
    'relationship': 'relationship',
    'since': 'since',
    'name_helpee': 'name',
    'gender_helpee': 'gender',
    'birth_date_helpee': 'birth_date',
    'helpee_bio': 'helpee_bio',
    'occupation_helpee': 'occupation',
    'disease': 'disease',
    'photo_url_helpee': 'photo_url',
    'number_helpee': 'number',
    'street_helpee': 'street',
    'zip_code_helpee': 'zip_code',
    'city_helpee': 'city',
    'latitude_helpee': 'latitude',
    'longitude_helpee': 'longitude',
}

caregiver_columns = {
    'caregiver_id': 'caregiver_id',
    'name_merged': 'name',
    'photo_url_merged': 'photo_url',
    'birth_date_merged': 'birth_date',
    'gender_merged': 'gender',
    'occupation_merged': 'occupation',
    'user_id': 'user_id',
    'experience': 'experience',
    'mental_load': 'mental_load',
    'helper_bio': 'helper_bio',
    'number_merged': 'number',
    'street_merged': 'street',
    'zip_code_merged': 'zip_code',
    'city_merged': 'city',
    'longitude_merged': 'longitude',
    'latitude_merged': 'latitude',
}



In [6]:
(
    caregiver_address_df.merge(
        relationship_df,
        on="caregiver_id",
        how="left",
        suffixes=["_caregiver", "_relationship"],
    ).merge(
        helpee_add_df,
        on="helpee_id",
        how="left",
        suffixes=["_merged", "_helpee"],
    ).groupby(list(caregiver_columns.keys()), dropna=False)
    .apply(
        lambda x: (
            x.loc[~pd.isna(x["helpee_id"]), list(helpee_columns.keys())]
            .rename(helpee_columns, axis=1)
            .to_dict(orient="records")
        )
    )
    .reset_index()
    .rename(caregiver_columns, axis=1)
    .rename({0: 'helpees'}, axis=1)
    .to_dict(orient="records")
)

[{'caregiver_id': '1',
  'name': 'Pierre',
  'photo_url': nan,
  'birth_date': '19850427',
  'gender': 'M',
  'occupation': 'active',
  'user_id': nan,
  'experience': 'new',
  'mental_load': 'low',
  'helper_bio': "Je suis Pierre, très beau, et m'occupe de ma maman, elle aussi très belle.",
  'number': '55',
  'street': 'rue la Fontaine',
  'zip_code': '92260',
  'city': 'Fontenay-aux-Roses',
  'longitude': '48.78602961813197',
  'latitude': '2.283740146208942',
  'helpees': []},
 {'caregiver_id': '2',
  'name': 'Marc',
  'photo_url': nan,
  'birth_date': '19850815',
  'gender': 'M',
  'occupation': 'active',
  'user_id': nan,
  'experience': 'medium',
  'mental_load': 'high',
  'helper_bio': "Je m'appelle Marc et je suis déplaisant à regarder.",
  'number': '67',
  'street': 'cours Gambetta',
  'zip_code': '13100',
  'city': 'Aix en Provence',
  'longitude': '43.519119711543965',
  'latitude': ' 5.462731655722142',
  'helpees': [{'helpee_id': '1',
    'relationship': 'mother',
    's

In [7]:
temp = caregiver_address_df.merge(
        relationship_df,
        on="caregiver_id",
        how="left",
        suffixes=["_caregiver", "_relationship"],
    ).merge(
        helpee_add_df,
        on="helpee_id",
        how="left",
        suffixes=["_merged", "_helpee"],
    )


In [8]:
temp.groupby(list(caregiver_columns.keys()), dropna=False).count().columns.duplicated()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [9]:
(            
temp.loc[~pd.isna(temp["helpee_id"]), helpee_columns.keys()]
            .rename(helpee_columns, axis=1)
            # .to_dict(orient="records")
)

,helpee_id,relationship,since,name,gender,birth_date,helpee_bio,occupation,disease,photo_url,number,street,zip_code,city,latitude,longitude
1,1,mother,20810115,Daniele,F,19570315,Ma mamounette chérie me donne souvent des choc...,retired,Maman poule,NaN,104,rue des amours d'antan,34980,Saint Gély du Fesc,3.8064635557728814,43.70153201752642
2,2,father,20821206,Jean-Marc,M,19540101,Mon papounet est un excellent camarade !,retired,Papa poulet,NaN,19,l'orée du bois,57580,Rémilly,6.384466569400809,49.013887388856425
